In [2]:
import sys 
from scipy.interpolate import interp1d
from scipy.interpolate import splrep as splrep

sys.path.append('/home/ak/Documents/Research/master/hsmm_core/')
sys.path.append('/home/ak/Documents/Research/master/')
import numpy as np
import pandas as pd
import os
from pandas.tseries.offsets import BDay
from hsmm_core.hmm import hmm_engine
from hsmm_core.observation_models import ExpIndMixDiracGauss
from hsmm_core.data_utils import load_data, TradingHours
import datetime as dt


In [3]:
no_states = 2
sigmas = [0.05, 0.002] # fast and slow
# Duration is measured in seconds for now (to be revised). lambda units are seconds^{-1}
# so here we consider

lambdas = [1./35., 1/20.]
weights = [0.1, 0.2]

obs_model = ExpIndMixDiracGauss(no_states)
obs_model.set_up_initials(priors={'sigmas': sigmas, 'lambdas': lambdas, 'weights': weights})

hmm_ = hmm_engine(obs_model, no_states)

# set up some priors
tpm = np.array([[0.45, 0.55], [0.8, 0.2]])
pi = np.array([0.4, 0.6])
hmm_.set_up_initials(priors={'tpm': tpm, 'pi': pi})

In [4]:
def states_from_fixed_ratios(ratios, total_length):
    states = np.array([], dtype=np.int64) 
    ratios_ids = np.arange(len(ratios))
    rng = np.random.RandomState(345)
    while len(states) < total_length: 
        
        ratio = ratios[rng.choice(ratios_ids)] 
        #print ratio
        states = np.append(states, np.append(np.repeat(0, 100*ratio[0]), np.repeat(1, 100*ratio[1]))) 
        #print len(states)
    return states

In [7]:

# Lets fix a ratio of 20:80 and sample some fixed states 
state_ratios = np.array([[0.2, 0.05], [0.4, 0.1], [0.8, 0.2]]) 

# Lets fix a ratio of 20:80 and sample some fixed states 
state_ratios = np.array([[0.2, 0.05], [0.4, 0.1], [0.8, 0.2]]) 
Ts = [20,50,100,200, 400,500, 700, 900,1000, 1200, 1500]
M = 10000

all_errors = np.zeros((len(Ts), M))


In [8]:
len(Ts)

11

In [9]:
###create sequences###
for i_T, T in enumerate(Ts):
    rng = np.random.RandomState(12345)
    states = states_from_fixed_ratios(state_ratios, T)
    
    for m in xrange(M):
        random_states = hmm_.sample_states(rng=rng, length=M)
        observation_points = obs_model.sample_data(M, rng=rng, state=states)
        viterbi_inferred = hmm_.map_metrics(observation_points)['viterbi_optimal_state_seq'] 
        all_errors[i_T, m] = np.sum((viterbi_inferred - states)**2)/float(T) 
        # The first duration is always zero
        observation_points[0, 0] = 0.

In [25]:

error_means_wrt_m = all_errors_df.mean(axis=1) 
final_errors = np.mean((df_all_errors - error_means_wrt_m[:, np.newaxis])**2, axis=1)

ValueError: Unable to coerce to DataFrame, shape must be (6, 10000): given (6, 1)

In [8]:
all_errors_df =pd.DataFrame(all_errors)

In [9]:
df_observation_points=pd.DataFrame(observation_points)
df_all_errors = pd.DataFrame(all_errors)
df_Ts= pd.DataFrame(Ts)

In [31]:
df_all_errors

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.200000,0.170000,0.230000,0.200000,0.200000,0.210000,0.190000,0.200000,0.220000,0.220000,...,0.190000,0.230000,0.180000,0.230000,0.160000,0.210000,0.160000,0.210000,0.150000,0.210000
1,0.205000,0.217500,0.250000,0.225000,0.230000,0.245000,0.227500,0.207500,0.227500,0.235000,...,0.250000,0.232500,0.215000,0.222500,0.227500,0.230000,0.242500,0.245000,0.250000,0.230000
2,0.187143,0.228571,0.191429,0.200000,0.195714,0.211429,0.201429,0.182857,0.197143,0.201429,...,0.201429,0.201429,0.208571,0.201429,0.200000,0.200000,0.198571,0.225714,0.187143,0.191429
3,0.182222,0.202222,0.190000,0.185556,0.201111,0.186667,0.182222,0.191111,0.195556,0.191111,...,0.203333,0.186667,0.191111,0.191111,0.192222,0.197778,0.196667,0.200000,0.197778,0.203333
4,0.189167,0.204167,0.202500,0.185000,0.198333,0.208333,0.196667,0.203333,0.202500,0.205833,...,0.197500,0.194167,0.199167,0.193333,0.198333,0.192500,0.186667,0.202500,0.207500,0.200833
5,0.180667,0.193333,0.207333,0.192000,0.197333,0.187333,0.205333,0.196667,0.178000,0.196000,...,0.190667,0.194000,0.192667,0.188667,0.182667,0.192000,0.186667,0.182667,0.186000,0.188667


In [35]:
all_errors =all_errors_df.values

In [36]:
np.mean(all_errors, axis=1)

array([0.192617  , 0.229269  , 0.20014414, 0.19300311, 0.19724108,
       0.19315133])

In [29]:
main_path = '/home/ak/Documents/Data/features_models/'

In [16]:
os.listdir(main_path)

['models',
 'observation_points.csv',
 'all_errors.csv',
 'features',
 'predictions',
 'synthetic_study',
 'Ts.csv',
 'observation_points_2.csv',
 'labels']

In [15]:
import pandas as pd
# df_observation_points = pd.DataFrame(observation_points)
obs_points= 'observation_points.csv'
all_errors = 'all_errors.csv'
Ts_file='Ts.csv'
# all_errors_df =pd.read_csv(main_path+all_errors)
# obs_points_df = pd.read_csv(main_path+obs_points)
df_observation_points.to_csv(os.path.join(main_path, obs_points), index=False)
df_all_errors.to_csv(os.path.join(main_path, all_errors), index=False)
df_Ts.to_csv(os.path.join(main_path, Ts_file), index= False)

In [ ]:
all_errors_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
total_error_fixedT = np.cumsum(all_errors_df.values, axis=1)
total_error_fixedT /= np.arange(1, M+1)
sns.set()
# plt.plot(total_error_fixedT/np.arange(M))
error_T100 = total_error_fixedT[5]
error_T700 = total_error_fixedT[11] 
error_T1500 = total_error_fixedT[-1] 
with plt.style.context('ggplot'):
    # Set an aspect ratio
    width, height = plt.figaspect(0.68)
    fig = plt.figure(figsize=(width,height), dpi=400)
    fig.patch.set_facecolor('white')
    #fig = plt.figure(figsize=(12,8))
    plt.plot(error_T100, color='grey',linestyle='--', label='T100')
    plt.plot(error_T700, color='blue',linestyle=':', label='T700')
    plt.plot(error_T1500, color='green',linestyle='-', label='T1500')

plt.legend(loc='upper right')
plt.xlabel('Sequence Length')
plt.ylabel('Error')
plt.show()

In [ ]:
import numpy as np
error_means_wrt_m = np.mean(all_errors,axis=1) 
final_errors = np.mean((all_errors - error_means_wrt_m[:, np.newaxis])**2, axis=1)

In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

In [ ]:
error_means_wrt_m = np.mean(all_errors,axis=1) 
final_errors = np.mean((all_errors - error_means_wrt_m[:, np.newaxis])**2, axis=1)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111) 
fig.subplots_adjust(top=0.85)
ax.set_ylabel('MC - error', fontsize=14)
ax.set_xlabel('T', fontsize=14)
ax.text(1000, .00025, 'Using 1e4 batches', fontsize=14) 
f2 = interp1d(Ts, final_errors, kind='linear')
f3=interpolate.splrep(Ts, final_errors, s=2)
#ynew = interpolate.splev(xnew, f3, der=1)
# ax.plot(Ts, final_errors,'--',color='r')

# #xnew = np.linspace(1, 1500, num=11, endpoint=True)
# #ax.plot(xnew, f2(xnew))
# #ax.plot(xnew, ynew)
# #import matplotlib.pyplot as plt
# plt.plot(x, y, 'o', xnew, f(xnew), '-', xnew, f2(xnew), '--')
# ax.axis([0, 1500, 0., 0.0003] ) ;

In [ ]:
f3

In [ ]:
from scipy.interpolate import interp1d
from scipy import interpolate

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111) 
fig.subplots_adjust(top=0.85)
ax.set_ylabel('MC - error', fontsize=14)
ax.set_xlabel('T', fontsize=14)
ax.text(1000, .00025, 'Using 1e4 batches', fontsize=14) 
f2 = interp1d(Ts, final_errors, kind='linear')
f3=interpolate.splrep(Ts, final_errors, s=2)
xnew = np.linspace(1, 1500, num=11, endpoint=True)
ynew = interpolate.splev(xnew, f3, der=1)
ax.plot(Ts, final_errors,'--',color='r')


ax.plot(xnew, f2(xnew))
ax.plot(xnew, ynew)
#import matplotlib.pyplot as plt
# plt.plot(x, y, 'o', xnew, f(xnew), '-', xnew, f2(xnew), '--')
ax.axis([0, 1500, 0., 0.0003] ) ;
tck, u = splrep(pts.T, u=None, s=0.0, per=1) 
u_new = np.linspace(u.min(), u.max(), 1000) 
x_new, y_new = splev(u_new, tck, der=0)

plt.plot(pts[:,0], pts[:,1], 'ro') 
plt.plot(x_new, y_new, 'b--') 
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111) 
fig.subplots_adjust(top=0.85)
ax.set_ylabel('MC - error', fontsize=14)
ax.set_xlabel('T', fontsize=14)
ax.text(1000, .00025, 'Using 1e4 batches', fontsize=14) 
f2 = interp1d(Ts, final_errors, kind='linear')
f3=interpolate.splrep(Ts, final_errors, s=2)

xnew = np.linspace(1, 1500, num=11, endpoint=True)
ynew = interpolate.splev(xnew, f3, der=1)
ax.plot(Ts, final_errors,'--',color='r')

ax.plot(xnew, f2(xnew))
ax.plot(xnew, ynew)
import matplotlib.pyplot as plt
plt.plot(x, y, 'o', xnew, f(xnew), '-', xnew, f2(xnew), '--')
ax.axis([0, 1500, 0., 0.0003] ) ;

In [ ]:
np.linspace(1, 1500, num=11, endpoint=True)